In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!curl https://colab.chainer.org/install | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1553  100  1553    0     0  12386      0 --:--:-- --:--:-- --:--:-- 12325
sh: 9: nvidia-smi: not found
********************************************************************************
GPU is not enabled!
Open "Runtime" > "Change runtime type" and set "Hardware accelerator" to "GPU".
********************************************************************************


In [ ]:
!pip install chainer

In [ ]:
!pip install cupy-cuda12x

In [ ]:
!pip install mpi4py

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from cupyx.profiler import benchmark
import timeit
from timeit import default_timer

ImportError: ignored

# Saxpy

In [ ]:
def saxpy(x, y, alpha):
  return alpha*x + y

In [ ]:
@cp.fuse(kernel_name='saxpy_cp')
def saxpy_cp(x, y, alpha):
  return alpha*x + y

In [ ]:
N = 100
x_np = np.random.rand(N, N).astype(np.float32)
y_np = np.random.rand(N, N).astype(np.float32)
alpha_np = np.random.rand()

In [ ]:
%%timeit
saxpy(x_np, y_np, alpha_np)

In [ ]:
N = 100
x_cp = cp.random.rand(N, N).astype(cp.float32)
y_cp = cp.random.rand(N, N).astype(cp.float32)
alpha_cp = cp.random.rand()

In [ ]:
%%timeit
saxpy_cp(x_cp, y_cp, alpha_cp)
cp.cuda.Device(0).synchronize()

In [ ]:
benchmark(saxpy, (x_cp, y_cp, alpha_cp), n_repeat=20, n_warmup=1)

# Key differences between CuPy and NumPy is that CuPy allows for the seamless transfer of arrays between the CPU and GPU.

# Function saxpy that runs on GPU using cupy is provided.

In [ ]:
array_size, comp_time_np, comp_time_cp = [], [], []
N = list(map(lambda x: 10**x, range(0, 8)))

for i in N:
  ### NumPy
  start = default_timer()
  x_np = np.random.rand(i).astype(np.float32)
  y_np = np.random.rand(i).astype(np.float32)
  alpha_np = np.random.rand()
  saxpy(x_np, y_np, alpha_np)
  comp_time_np.append(default_timer() - start)

  ### CuPy
  start = default_timer()
  x_cp = cp.random.rand(i).astype(cp.float32)
  y_cp = cp.random.rand(i).astype(cp.float32)
  alpha_cp = cp.random.rand()
  saxpy_cp(x_cp, y_cp, alpha_cp)
  comp_time_cp.append(default_timer() - start)

# Plot computation time of numpy and cupy implementations of saxpy

In [ ]:
fig = plt.figure(layout='constrained', figsize=(10, 4))
axs = fig.subplots(1, 2, sharex=True)


for ax in axs:
    ax.set_xlabel("size of arrays")
    ax.set_xscale('log', base=10)
    ax.grid()


axs[0].plot(N, comp_time_np, label='NumPy')
axs[0].plot(N, comp_time_cp, label="CuPy" )
axs[0].set_ylabel("computation time, s")
axs[0].legend()


axs[1].plot(N,
            list(map(lambda x, y: x/y,comp_time_np, comp_time_cp)),
            marker='o'
            )
axs[1].set_ylabel("speed up")

# Bifurcation diagram

### Bifurcation map is performed using numpy arrays

In [ ]:
x0 = 0.1
r_min, r_max = 1, 5.0
num_steps = 1000
n, k = 1000, 500

In [ ]:
from scipy.integrate import quad

def logistic_map(x, r):
    return r * x * (1. - x)

In [ ]:
def bifurcation_diagram_np(x0, r_min, r_max, num_steps, n, k):
    r_list = np.linspace(r_min, r_max, num_steps)
    bifurcations = []

    for r in r_list:
        x = x0
        for i in range(n + k):
            if i >= k:
                bifurcations.append([r, x])
            x = logistic_map(x, r)


    bifurcations = np.array(bifurcations)
    return bifurcations

bifurcations = bifurcation_diagram_np(x0, r_min, r_max, num_steps, n, k)

In [ ]:
plt.plot(bifurcations[:,0], bifurcations[:,1], 'o', markersize=0.2)
plt.xlabel('r')
plt.ylim(0, 1)
plt.ylabel('x')
plt.title('Numpy')
plt.show()

### Bifurcation map is performed using cupy arrays

In [ ]:
@cp.fuse(kernel_name='logistic_map_cp')
def logistic_map_cp(x, r):
    return r * x * (1. - x)

In [ ]:
@cp.fuse(kernel_name='bifurcation_diagram_cp')
def bifurcation_diagram_cp(x0, r_min, r_max, num_steps, n, k):
    r_list = cp.linspace(r_min, r_max, num_steps)
    bifurcations = []

    for r in r_list:
        x = x0
        for i in range(n + k):
            if i >= k:
                bifurcations.append([r, x])
            x = logistic_map_cp(x, r)


    bifurcations = cp.array(bifurcations)
    return bifurcations

bifurcations_cp = bifurcation_diagram_cp(x0, r_min, r_max, num_steps, n, k)

In [ ]:
plt.plot(bifurcations_cp[:,0].get(),
         bifurcations_cp[:,1].get(), 'o', markersize=0.2)
plt.xlabel('r')
plt.ylim(0, 1)
plt.ylabel('x')
plt.show()

### Bifurcation map is performed using MPI

In [ ]:
%%writefile bifurcation_diagram_MPI.py

from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

x0 = 0.1
r_min, r_max = 1, 5.0
num_steps = 10000
n, k = 1000, 500

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

start_time = MPI.Wtime()

local_n = num_steps // size
r_min_local = r_min + rank * local_n * (r_max-r_min) / num_steps
r_max_local = r_min_local + local_n * (r_max-r_min) / num_steps
local_r_values = np.linspace(r_min_local, r_max_local, local_n)


local_bifurcations = []

def logistic_map(x, r):
    return r * x * (1. - x)

for r in local_r_values:
    x = x0
    for i in range(n + k):
        if i >= k:
            local_bifurcations.append([r, x])
        x = logistic_map(x, r)


all_bifurcation = np.empty(local_n * size, dtype=np.float32)
all_bifurcation = comm.gather(local_bifurcations, root=0)

end_time = MPI.Wtime()
mpi_time = end_time - start_time

MPI.Finalize()

if rank == 0:

    data = np.concatenate(all_bifurcation)
    print(mpi_time)

    plt.scatter(data[:,0], data[:,1], s=1)
    plt.xlabel('r')
    plt.ylabel('x')
    plt.savefig('Bif_mpi.png')

In [ ]:
!mpirun -n 4 --allow-run-as-root --oversubscribe python bifurcation_diagram_MPI.py

In [ ]:
start = default_timer()
bifurcation_diagram_cp(x0, r_min, r_max, num_steps, n, k)
serial_time_cp = (default_timer() - start)

In [ ]:
start = default_timer()
bifurcation_diagram_np(x0, r_min, r_max, num_steps, n, k)
serial_time_np = (default_timer() - start)

# Plot computation time of CPU, CPU parallel, and GPU implementations.

In [ ]:
def mpi(N):
  mpi_time = !mpirun -n {N} --allow-run-as-root --oversubscribe python bifurcation_diagram_MPI.py
  return mpi_time

num_processes = [1, 2, 3, 4]
parallel_time, serial_np, serial_cp = [], [], []
serial_np = [serial_time_np]*len(num_processes)
serial_cp = [serial_time_cp]*len(num_processes)

for N in num_processes:
    parallel_time.append(np.float32(mpi(N)))

In [ ]:
plt.plot(num_processes, serial_np, marker='o', label='CPU')
plt.plot(num_processes, parallel_time, marker='o', label='CPU parallel')
plt.plot(num_processes, serial_cp, marker='o', label='GPU')
plt.xlabel('Number of Processes')
plt.ylabel('Computation time')
plt.title('Computation time vs. Number of Processes')
plt.legend()
plt.show()